#Firstly, we need to dowload additional packages

In [ ]:
!pip install transformers
!pip install numpy
!pip install pandas
!pip install scipy
!pip install tqdm

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#Now I can create my model
####Available tasks: 
emoji, emotion, hate, irony, offensive, sentiment

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [ ]:

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
def predict_text(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores

def print_scores(scores):
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")


# Sentiment analysis

## Calculating sentiment for each individual opinion

In [ ]:
import pandas as pd

df = pd.read_csv('indeed_reviews.csv', sep=';', index_col=0)

print(df.head())

   rate  ...        date
1   5.0  ...  2021-05-05
2   5.0  ...  2021-05-01
3   5.0  ...  2021-01-31
4   5.0  ...  2020-12-30
5   5.0  ...  2020-11-25

[5 rows x 7 columns]


In [ ]:
df['combined_review'] = df.apply(lambda row: f'{row.title} - {row.review}', axis=1)
df[
   ['negative_score', 'neutral_score', 'positive_score']
] = df['combined_review'].progress_apply(lambda review: pd.Series(predict_text(review)))

100%|██████████| 3461/3461 [15:51<00:00,  3.64it/s]


In [ ]:
df['year'] = df['date'].apply(lambda x: x.split('-')[0])

In [ ]:
result_df = df[['company', 'year', 'date', 'combined_review', 'rate','negative_score', 'neutral_score', 'positive_score']]

result_df.to_csv('indeed_reviews_sentiment.csv', sep=';')

## Combining opinins together

In [ ]:
summary = result_df[['company', 'year', 'rate','negative_score', 'neutral_score', 'positive_score']]

summary_grouped = summary.groupby(by=['company', 'year']).mean()

summary_grouped.to_csv('summary.csv', sep=';')

print(summary_grouped)

                                   rate  ...  positive_score
company                  year            ...                
BMW-Group_reviews        2020  3.459854  ...        0.549161
                         2021  3.465116  ...        0.679373
BP_reviews               2020  3.779070  ...        0.639799
                         2021  3.450980  ...        0.411694
Bharat-Petroleum_reviews 2020  4.564103  ...        0.845114
                         2021  5.000000  ...        0.954536
Dcc_reviews              2020  4.000000  ...        0.188387
                         2021  4.000000  ...        0.934391
Equinor_reviews          2020  3.714286  ...        0.647984
                         2021  3.333333  ...        0.527555
Faurecia_reviews         2020  2.976190  ...        0.416184
                         2021  2.806452  ...        0.266011
Hindustan-Petrol_reviews 2020  4.285714  ...        0.798389
                         2021  4.000000  ...        0.946953
Honda_reviews           